[![N|BWF](http://bwfbadminton.com/wp-content/themes/fansite/img/banner-bw-tv.jpg)](http://bwfbadminton.com/)
# Scrap Data BWF for Predict Indonesia Open 2018


Collection data profile players and history match for prediction the winning of **Indonesia Open 2018**

Using **BeautifulSoap** and **Requests** python module for collection data

Source : **bwfworldtour.com**

In [45]:
import requests
import scrapy
import io, json
import mpu.io
from bs4 import BeautifulSoup
from tqdm import tqdm
from pprint import pprint

In [24]:
url = """http://bwfworldtour.com/rankings/?id=9&cat_id=57&ryear=2018&week=26&page_size=1000&page_no=1"""

In [25]:
data = requests.get(url=url)

In [26]:
html = data.text

In [27]:
soup = BeautifulSoup(html, "html5lib")

## Function Get Profile Players

In [28]:
def get_profile(url=None):
    html = requests.get(url=url).text
    soup = BeautifulSoup(html, "html5lib")
    ranking_world = int(soup.select_one(".player-world-rank > .ranking-number").text.strip())
    ranking_number = int(soup.select_one(".player-wtr-rank > .ranking-number").text.strip())
    player_wins = int(soup.select_one(".player-wins > .large").text.strip())
    player_age = int(soup.select_one(".player-age > .large").text.strip())
    player_dob = soup.select(".player-age > span")[2].text.strip()
    player_handed = soup.select(".player-handed > span")[1].text.strip()
    player_height = soup.select(".player-stats > p > span")[1].text.strip()
    result = {
        "ranking_world": ranking_world,
        "ranking_number": ranking_number,
        "player_wins": player_wins,
        "player_age": player_age,
        "player_dob": player_dob,
        "player_handed": player_handed,
        "player_height":player_height
    }
    return result

## Function Get Head To Head Players

In [29]:
def get_h2h(url=None):
    html = requests.get(url=url).text
    soup = BeautifulSoup(html, "html5lib")
    
    player1 = soup.select(".info-player1 > .info-name > .name > a")[0].text.strip()
    player2 = soup.select(".info-player2 > .info-name > .name > a")[0].text.strip()
    score_player1 = 0
    score_player2 = 0        
    
    data = soup.find_all("div", {"class":"player-result-score"})
    for d in data:
        if d.text.split(' ')[0].strip() == "W":
            score_player1 += 1
        else:
            score_player2 += 1
            
    
    jumlah_pertemuan = score_player1 + score_player2
    test = soup.find_all("span", {"class":"score"})
    hasil_score = []
    for i in test:
        hasil_score.append(i.text.split(' '))

    result = {
        "player1": player1,
        "player2": player2,
        "score_player1": score_player1,
        "score_player2": score_player2,
        "hasil_score": hasil_score
    }
                  
    return result

## Function Get CareerSingles

In [30]:
def get_careerSingles(url=None):
    html = requests.get(url=url).text
    soup = BeautifulSoup(html, "html5lib")
        
    banyak_bermain = soup.find_all("td")[1].get_text()
    menang = soup.find_all("td")[2].get_text()
    kalah = soup.find_all("td")[3].get_text()
    seri = soup.find_all("td")[4].text.strip()
    pendapatan = int(soup.find_all("td")[5].text.strip('').replace(',',''))
        
    result = {
        "banyak_bermain": banyak_bermain,
        "menang": menang,
        "kalah": kalah,
        "seri": seri,
        "pendapatan": pendapatan
    }
    
    return result

## Function Get CareerDoubles

In [31]:
def get_careerDoubles(url=None):
    html = requests.get(url=url).text
    soup = BeautifulSoup(html, "html5lib")
        
    banyak_bermain = soup.find_all("td")[13].get_text()
    menang = soup.find_all("td")[14].get_text()
    kalah = soup.find_all("td")[15].get_text()
    seri = soup.find_all("td")[16].text.strip()
    pendapatan = int(soup.find_all("td")[5].text.strip('').replace(',',''))
    
    result = {
        "banyak_bermain": banyak_bermain,
        "menang": menang,
        "kalah": kalah,
        "seri": seri,
        "pendapatan": pendapatan
    }
    
    return result

In [59]:
rankings = []
res = soup.select(".rankings-table tr")[:5]

id_players = []

for i in tqdm(range(len(res))):
    try:
        x = res[i+1]
        r = x.find_all("td")
        if len(r) > 1:
            id_player = int(r[2].find("a")["href"].split("/")[4])
            url = r[2].find("a")["href"]
            additional = get_profile(url)
            url_h2h = url + ("/head-to-head-analysis")
            headTohead = get_h2h(url_h2h)
            
            careerSingles = get_careerSingles(url)
            careerDoubles = get_careerDoubles(url)
            
            id_players.append(id_player)
            rank = {
                
#                 Profile Players
                "rank": int(r[0].text.strip()),
                "country": r[1].text.strip(),
                "name": r[2].text.strip(),
                "id_player": id_player,
                "url": url,
                "change": r[3].text.strip(),
                "win": int(r[4].text.strip().split("/")[0]),
                "lose": int(r[4].text.strip().split("/")[1]),
                "price": int(r[5].text.strip().replace(",", "").replace("$", "")),
                "point": int(r[6].text.strip().replace(",", "")),
                
                "ranking_world": additional['ranking_world'],
                "ranking_world_tour": additional['ranking_number'],
                "ranking_world": additional['ranking_world'],
                "player_wins": additional['player_wins'],
                "player_age": additional['player_age'],
                "player_handed": additional['player_handed'],
                "player_height": additional['player_height'],
                
                
#                 Career Single Players
                "banyak_bermain(CareerSingles)":careerSingles['banyak_bermain'],
                "menang(CareerSingles)":careerSingles['menang'],
                "kalah(CareerSingles)":careerSingles['kalah'],
                "seri(CareerSingles)":careerSingles['seri'],
                "pendapatan(CareerSingles)":careerSingles['pendapatan'],
                
#                 Career Double Players
                "banyak_bermain(CareerDoubles)":careerDoubles['banyak_bermain'],
                "menang(CareerDoubles)":careerDoubles['menang'],
                "kalah(CareerDoubles)":careerDoubles['kalah'],
                "seri(CareerDoubles)":careerDoubles['seri'],
                "pendapatan(CareerDoubles)":careerDoubles['pendapatan'],
                
                
#                 History Head To head
                "player1": headTohead['player1'],
                "player2": headTohead['player2'],
                "score_player1": headTohead['score_player1'],
                "score_player2": headTohead['score_player2'],
                "hasil_score": headTohead['hasil_score']
            }
            rankings.append(rank)
    except:
        pass
    
data = rankings
with open('prediksi_badminton.json', 'w') as f:
    json.dump(data, f)

100%|██████████| 5/5 [00:12<00:00,  2.60s/it]


In [210]:
for p in id_players:
    url = """http://bwfworldtour.com/player/14587/tommy-sugiarto/head-to-head-analysis/?event=single&team2_player1={players}""".format(players=p)
    result = get_h2h(url)
    print(result)
    print()

{'player1': 'TommySUGIARTO', 'player2': 'TommySUGIARTO', 'score_player1': 0, 'score_player2': 0, 'hasil_score': []}

{'player1': 'TommySUGIARTO', 'player2': 'LINDan', 'score_player1': 2, 'score_player2': 6, 'hasil_score': [['18-21,', '14-21'], ['17-21,', '17-21'], ['19-21,', '21-8,', '21-16'], ['0-2'], ['21-17,', '15-21,', '21-17'], ['10-21,', '11-21'], ['15-21,', '15-21'], ['12-21,', '19-21']]}



## Result Players Ranking

In [ ]:
rankings